In [75]:
import pandas as pd
import datetime
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore") 
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score
from scipy import stats
from sklearn.metrics import precision_score
from pprint import pprint

In [76]:
Filepath = 'C:/Users/ChowdhKa/Downloads/Kakoli Masters/MRP/All File/'

In [77]:
# Read cluster file
fields = ['Channel', 'clusterNum']
clusterFile = pd.read_csv(Filepath+'All_Cluster.csv', skipinitialspace=True, usecols=fields)
clusterFile = clusterFile[(clusterFile['clusterNum'] == 2)]

In [78]:
#read layer 3 data csv
df=pd.read_csv(Filepath+'OctDataSmall.csv')
df.columns = ['StartDate', 'EndDate', 'Channel', 'Pwr_dbm','Snr', 'Occupancy']
df['StartDate'] = pd.to_datetime(df['StartDate'])
df['EndDate'] = pd.to_datetime(df['EndDate'])
df['StartDate'] = df['StartDate'].values.astype('<M8[h]')
df['EndDate'] = df['EndDate'].values.astype('<M8[h]')
df.head()

,StartDate,EndDate,Channel,Pwr_dbm,Snr,Occupancy
0,2016-10-06,2016-10-06 01:00:00,56303,-83.293709,21.397917,100.0
1,2016-10-06,2016-10-06 01:00:00,56304,-63.193748,41.497952,100.0
2,2016-10-06,2016-10-06 01:00:00,56305,-31.322813,73.445999,100.0
3,2016-10-06,2016-10-06 01:00:00,56306,-64.096245,41.568588,100.0
4,2016-10-06,2016-10-06 01:00:00,56307,-83.440269,21.249647,100.0


In [79]:
#putting dataframe in another dataframe 
dataf = df

In [80]:
# Take the data frame from R file
df_merged=pd.read_csv(Filepath+'testcsv.csv',delimiter="\t")
#print(df_merged)
df_merged.columns = ['Channel','StartDate', 'EndDate', 'Pwr_dbm','Snr', 'Occupancy','per_read']
df_merged['StartDate'] = pd.to_datetime(df_merged['StartDate'])
df_merged['EndDate'] = pd.to_datetime(df_merged['EndDate'])
df_merged['StartDate'] = df_merged['StartDate'].values.astype('<M8[h]')
df_merged['EndDate'] = df_merged['EndDate'].values.astype('<M8[h]')
df_merged.head()

,Channel,StartDate,EndDate,Pwr_dbm,Snr,Occupancy,per_read
1,10,2016-10-07 08:00:00,2016-10-07 09:00:00,-91.500000,11.700000,0.074493,82.82504
2,10,2016-10-21 21:00:00,2016-10-21 22:00:00,-90.774193,12.806452,0.466165,82.82504
3,10,2016-10-11 01:00:00,2016-10-11 02:00:00,-91.592590,11.444445,0.203912,82.82504
4,10,2016-10-21 12:00:00,2016-10-21 13:00:00,-91.340424,11.851064,0.352800,82.82504
5,10,2016-10-20 23:00:00,2016-10-21 00:00:00,-92.064514,11.580646,0.237639,82.82504


In [81]:
# Taking only the required columns
df_merged = df_merged[['Channel','StartDate', 'EndDate', 'Occupancy']]

In [82]:
# JOIN df_merged and clusterFile on channels
clusterChannelDF = pd.merge(clusterFile, df_merged[['Channel', 'StartDate', 'EndDate', 'Occupancy']],on='Channel')
clusterChannelDF = clusterChannelDF.drop(['clusterNum'], 1)

In [83]:
#Getting the final channel list

channelList = clusterChannelDF.Channel.unique().tolist()
#channelList = np.random.choice(channel, 300, replace=False)

In [84]:
#channelList.tolist()

In [85]:
#channelList = [20750,22320,23200,28430,34680,34690,50267,15220,15530,15570,15820,16010,16130]#,
               #21340,24220,25410,53970,54006,54494,54620,56097,56293,56305,56306,56307]

In [86]:
#channelList = [20750,22320,23200,28430]
               #,34680,34690,50267,15220,15530,15570,15820,16010,16130,
               #21340,24220,25410,53970,54006,54494,54620,56097,56293,56305,56306,56307]

In [87]:
OriginalMergedDF = clusterChannelDF

In [88]:
def ProcessDF(channel, threshold):
    InterDF = OriginalMergedDF[(OriginalMergedDF.Channel == channel)][['StartDate', 'EndDate', 'Channel', 'Occupancy']]
    InterDF = InterDF.set_index('StartDate')
    InterDF = InterDF.sort_index()
    indexStart = InterDF.index[0]
    indexEnd = InterDF.index[-1]
    InterDF= InterDF.reindex(pd.date_range(start=indexStart, end=indexEnd, freq='1h'))
    shiftVales = 2
    InterDF['Occupancy1'] = InterDF['Occupancy'].shift()
    while shiftVales <=6:
        InterDF['Occupancy'+str(shiftVales)] = InterDF['Occupancy'+str(shiftVales-1)].shift()
        shiftVales=shiftVales+1
    # Convert index back to dataframe column
    InterDF.reset_index(level=0, inplace=True)
    InterDF.drop(InterDF.columns[[0,1]], axis=1, inplace=True)
    InterDF = InterDF.dropna(how='any', axis = 0)
    #InterDF['OccupancyLabel'] = np.where(InterDF['Occupancy'] > threshold, 1,0)
    InterDF['OccupancyLabel'] = pd.cut(InterDF['Occupancy'], [-np.inf, threshold, np.inf], labels=[0, 1])
    return InterDF

In [89]:
def GausNB(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    predicted = model.predict(X_test)
    confusion_NB = confusion_matrix(y_test, predicted, labels=[0,1])
    # Accuracy
    scoreNB = accuracy_score(y_test, predicted)
    # Sensitivity
    senseNB = confusion_NB[1,1]/(confusion_NB[1,0]+confusion_NB[1,1])
    # Specificity
    specificNB = confusion_NB[0,0]/(confusion_NB[0,0]+confusion_NB[0,1])
    # F-measure
    fmeasureNB = f1_score(y_test, predicted,average='weighted')
    # Precision
    precisionscoreNB = precision_score(y_test, predicted, average='weighted')

    return scoreNB, senseNB, specificNB, fmeasureNB, precisionscoreNB

In [90]:
def LogReg(X_train, y_train, X_test, y_test):
    reg = LogisticRegression()
    modfit = reg.fit(X_train, y_train)
    yPred = modfit.predict(X_test)
    confusion_LR = confusion_matrix(y_test, yPred, labels=[0,1])
    # Accuracy
    # scoreLR = modfit.score(X_test, y_test)
    scoreLR = (confusion_LR[0,0]+confusion_LR[1,1])/(confusion_LR[0,0]+confusion_LR[0,1]+confusion_LR[1,0]+confusion_LR[1,1])
    # Sensitivity
    senseLR = confusion_LR[1,1]/(confusion_LR[1,0]+confusion_LR[1,1])
    # Specificity
    specificLR = confusion_LR[0,0]/(confusion_LR[0,0]+confusion_LR[0,1])
    # F-measure
    fmeasureLR = f1_score(y_test, yPred, average='weighted')
    # Precision
    precisionscoreLR = precision_score(y_test, yPred, average='weighted')

    return scoreLR, senseLR, specificLR, fmeasureLR, precisionscoreLR

In [91]:
#print(k[1])

In [92]:
# Defining the threshold for class labeling
SaveDataDF = pd.DataFrame([])
t = 70
#list_threshold = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
#list_threshold = [40, 50, 60, 70]
#for t in list_threshold:
for channel in channelList:
    #print('Value of Threshold: ' + str(t))
    #print('Current channel is: '+ str(channel))

    ProcessedDF = ProcessDF(channel, t)
    ProcessedDFx = ProcessedDF[(ProcessedDF.Channel == channel)][['Channel', 'Occupancy1', 'Occupancy2', 'Occupancy3', 'Occupancy4', 'Occupancy5', 'Occupancy6']]
    ProcessedDFy = ProcessedDF[(ProcessedDF.Channel == channel)][['OccupancyLabel']] 
    #print(ProcessedDFy.OccupancyLabel.unique())
    #print(ProcessedDFy['OccupancyLabel'].value_counts())
    k = ProcessedDFy.groupby('OccupancyLabel').size()
    #print('k[0]: '+ str(k[0]))
    #print('k[1]: '+ str(k[1]))
    if (k[0] > 5 and k[1] > 5):
        RunIter = 1
        while RunIter <= 10:
            foldCounter = 1
            kc = KFold(n_splits = 10, shuffle=True)
            for train_x, test_x in kc.split(ProcessedDFx, ProcessedDFy):
                X_trainDF, X_testDF = ProcessedDFx.iloc[train_x], ProcessedDFx.iloc[test_x]
                y_trainDF, y_testDF = ProcessedDFy.iloc[train_x], ProcessedDFy.iloc[test_x]


                # Naive Bayes
                scoreNB, sensitivityNB, specificityNB, fmeasureNB, precisionNB = GausNB(X_trainDF, y_trainDF, X_testDF, y_testDF)
                #if len(y_trainDF.OccupancyLabel.unique()) > 1:
                # Logistic Regression
                scoreLR, sensitivityLR, specificityLR, fmeasureLR, precisionLR = LogReg(X_trainDF, y_trainDF, X_testDF, y_testDF)

                # PRINT ROW VALUE IN DATAFRAME- KEEP APPENDING
                SaveDataDF = SaveDataDF.append(pd.DataFrame({'Channel': channel, 'Threshold': t,
                                                             'RunCount': RunIter,'FoldCount': foldCounter,
                                                             'AccuracyLR': scoreLR, 'SensitivityLR': sensitivityLR,
                                                             'SpecificityLR': specificityLR, 'AccuracyNB': scoreNB,
                                                             'SensitivityNB': sensitivityNB, 'SpecificityNB': specificityNB,
                                                             'fmeasureNB': fmeasureNB, 'fmeasureLR': fmeasureLR,
                                                             'precisionLR': precisionLR, 'precisionNB': precisionNB
                                                            }, 
                                                            index=[0]), ignore_index=True)
            foldCounter = foldCounter + 1
            RunIter = RunIter + 1


In [93]:
#fmeasureLR

In [94]:
#ProcessedDFy.head()

In [95]:
ProcessedDFx.head()

,Channel,Occupancy1,Occupancy2,Occupancy3,Occupancy4,Occupancy5,Occupancy6
6,56422.0,7.678287,14.372973,16.154018,21.608527,31.603633,40.126535
7,56422.0,7.427550,7.678287,14.372973,16.154018,21.608527,31.603633
8,56422.0,10.939597,7.427550,7.678287,14.372973,16.154018,21.608527
9,56422.0,8.550129,10.939597,7.427550,7.678287,14.372973,16.154018
10,56422.0,10.559574,8.550129,10.939597,7.427550,7.678287,14.372973


In [96]:
SaveDataDF.head()

,AccuracyLR,AccuracyNB,Channel,FoldCount,RunCount,SensitivityLR,SensitivityNB,SpecificityLR,SpecificityNB,Threshold,fmeasureLR,fmeasureNB,precisionLR,precisionNB
0,0.959184,0.938776,830,1,1,0.000000,1.000000,1.000000,0.936170,70,0.939201,0.950886,0.920033,0.975510
1,0.959184,0.979592,830,1,1,0.500000,0.750000,1.000000,1.000000,70,0.952825,0.978246,0.960921,0.980035
2,0.897959,0.897959,830,1,1,0.000000,0.250000,0.977778,0.955556,70,0.868993,0.891231,0.841837,0.885685
3,0.959184,0.877551,830,1,1,0.666667,0.666667,0.978261,0.891304,70,0.959184,0.899258,0.959184,0.933916
4,0.959184,0.918367,830,1,1,0.000000,0.500000,1.000000,0.936170,70,0.939201,0.931085,0.920033,0.948073


In [97]:
SaveDataDF.to_csv(Filepath+'OutputData.csv', header=False,sep='\t', encoding='utf-8', index=False)

In [98]:
readResults = pd.read_csv(Filepath+'OutputData.csv', header=1, sep ='\t')

In [99]:
readResults.columns = ['AccuracyLR', 'AccuracyNB', 'Channel', 'FoldCount', 'RunCount', 
                    'SensitivityLR', 'SensitivityNB','SpecificityLR', 
                    'SpecificityNB','Threshold','FmeasureLR', 'FmeasureNB', 'PrecisionLR', 'PrecisionNB']

In [100]:
readResults.head()

,AccuracyLR,AccuracyNB,Channel,FoldCount,RunCount,SensitivityLR,SensitivityNB,SpecificityLR,SpecificityNB,Threshold,FmeasureLR,FmeasureNB,PrecisionLR,PrecisionNB
0,0.897959,0.897959,830,1,1,0.000000,0.250000,0.977778,0.955556,70,0.868993,0.891231,0.841837,0.885685
1,0.959184,0.877551,830,1,1,0.666667,0.666667,0.978261,0.891304,70,0.959184,0.899258,0.959184,0.933916
2,0.959184,0.918367,830,1,1,0.000000,0.500000,1.000000,0.936170,70,0.939201,0.931085,0.920033,0.948073
3,1.000000,0.938776,830,1,1,1.000000,1.000000,1.000000,0.937500,70,1.000000,0.956155,1.000000,0.984694
4,0.958333,0.854167,830,1,1,0.750000,0.750000,0.977273,0.863636,70,0.958333,0.877819,0.958333,0.920940


In [101]:
readResults.shape

(8098, 14)

In [102]:
len(SaveDataDF['AccuracyNB'])

8100

In [103]:
SaveDataDF.dtypes

AccuracyLR       float64
AccuracyNB       float64
Channel            int64
FoldCount          int64
RunCount           int64
SensitivityLR    float64
SensitivityNB    float64
SpecificityLR    float64
SpecificityNB    float64
Threshold          int64
fmeasureLR       float64
fmeasureNB       float64
precisionLR      float64
precisionNB      float64
dtype: object

In [104]:
pVal = stats.ttest_ind(SaveDataDF['AccuracyNB'][:800],SaveDataDF['AccuracyLR'][:800], nan_policy = 'propagate')
pValue = pVal[1]

In [105]:
pVal

Ttest_indResult(statistic=-26.381158090376097, pvalue=1.2805201154775418e-127)

In [106]:
# This code is commented and can be used when required to print the p-value
#pprint([(i, stats.ttest_ind(SaveDataDF['AccuracyNB'], SaveDataDF['AccuracyLR'][:i])[1])
#        for i in range(100,len(SaveDataDF['AccuracyLR']),100)])

In [107]:
readResults = readResults.drop(['FoldCount', 'RunCount'], 1)

In [108]:
PerThresPerChannelDF = readResults.groupby(['Channel','Threshold']).mean().add_suffix('_mean').reset_index()

In [109]:
PerThresPerChannelDF.head()

,Channel,Threshold,AccuracyLR_mean,AccuracyNB_mean,SensitivityLR_mean,SensitivityNB_mean,SpecificityLR_mean,SpecificityNB_mean,FmeasureLR_mean,FmeasureNB_mean,PrecisionLR_mean,PrecisionNB_mean
0,830,70,0.954190,0.915560,0.357476,0.656247,0.984858,0.928940,0.947958,0.930692,0.948595,0.954219
1,1370,70,0.975948,0.960485,0.000000,0.000000,0.998537,0.982744,0.965574,0.957704,0.955618,0.955300
2,2020,70,0.923044,0.832462,0.702504,0.786134,0.965454,0.841706,0.920901,0.848430,0.925437,0.884460
3,3860,70,0.903227,0.851233,0.624384,0.701559,0.960458,0.884082,0.899217,0.858657,0.904911,0.875965
4,9230,70,0.934643,0.812224,0.157649,0.546213,0.985485,0.831113,0.918371,0.855878,0.908939,0.920886


In [110]:
PerThresPerChannelDF['CountChan'] = 1
aggregator = {'AccuracyLR_mean' : 'mean',
              'AccuracyNB_mean' : 'mean',
              'SensitivityLR_mean': 'mean',
              'SensitivityNB_mean' : 'mean',
              'SpecificityLR_mean': 'mean',
              'SpecificityNB_mean' : 'mean',
              'FmeasureLR_mean':'mean',
              'FmeasureNB_mean':'mean',
              'PrecisionLR_mean': 'mean',
              'PrecisionNB_mean': 'mean',
              'CountChan': 'sum'}
FinalResultsDF = PerThresPerChannelDF.groupby('Threshold', as_index=False)['AccuracyLR_mean', 'AccuracyNB_mean', 
                                                                 'SensitivityLR_mean', 'SensitivityNB_mean', 
                                                                 'SpecificityLR_mean', 'SpecificityNB_mean',
                                                                 'PrecisionLR_mean','PrecisionNB_mean',
                                                                 'FmeasureLR_mean', 'FmeasureNB_mean'].agg(aggregator)


In [111]:
#PerThresPerChannelDF = PerThresPerChannelDF.drop(['Channel'], 1)
#FinalResultsDF = PerThresPerChannelDF.groupby(['Threshold']).mean().reset_index()
FinalResultsDF['Cluster'] = '2'
FinalResultsDF['pValue'] = pValue

In [112]:
FinalResultsDF

,Threshold,AccuracyLR_mean,AccuracyNB_mean,SensitivityLR_mean,SensitivityNB_mean,SpecificityLR_mean,SpecificityNB_mean,FmeasureLR_mean,FmeasureNB_mean,PrecisionLR_mean,PrecisionNB_mean,CountChan,Cluster,pValue
0,70,0.948458,0.878009,0.462043,0.731838,0.972968,0.878027,0.943943,0.899268,0.943976,0.93607,81,2,1.280520e-127


In [113]:
# Write the results to a CSV
FinalResultsDF.to_csv(Filepath+'Output/FinalResults.csv', header=False,sep='\t', index=False, mode = 'a')